# Implement Attention from Scratch
###  Problem Statement
Transformers are order-agnostic — they see tokens like goldfish: no sense of sequence. To inject **position awareness** into the model, we use **Sinusoidal Positional Embeddings**, where each position in the sequence gets a unique deterministic vector. These vectors are computed using sine and cosine waves at different frequencies.

Your task is to implement the sinusoidal position encoding mechanism from scratch using PyTorch — no cheating with built-ins from `fairseq` or Hugging Face.

---

###  Requirements

1. **Define the Sinusoidal Embedding Class**
   - Implement a `SinusoidalPositionalEmbedding` class inheriting from `nn.Module`.
   - Initialize with `max_seq_len` and `d_model`.
   - Create a tensor `pe` of shape `(max_seq_len, d_model)` filled with sine and cosine encodings:
     - `sin(position * ω)` for even indices
     - `cos(position * ω)` for odd indices

2. **Register as Buffer**
   - Use `self.register_buffer("pe", pe)` to store `pe` without treating it as a trainable parameter.

3. **Generate Encodings**
   - On calling `forward(x)`, return the slice of positional encodings matching the sequence length of `x`.

4. **Test the Embeddings**
   - Initialize the embedding class with `max_seq_len = 100` and `d_model = 64`.
   - Pass a sequence of length 50 to verify the returned shape is `(1, 50, 64)`.

---

### Constraints

- ✅ Do not use Hugging Face, Fairseq, or built-in PyTorch modules for position encoding.
- ✅ Ensure the `pe` tensor is not a trainable parameter.
- ✅ Support any sequence length up to `max_seq_len`.
- ❌ Do not inject these embeddings directly into token embeddings yet — this is just the embedding module.

---

<details>
  <summary>💡 Hint</summary>

  - Use `torch.arange(0, max_seq_len).unsqueeze(1)` to create position indices.
  - Compute frequencies with `torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))`.
  - Alternate `sin` and `cos` values for even and odd embedding dimensions.
  - When returning the embedding in `forward`, use `.unsqueeze(0)` to broadcast over the batch dimension.

</details>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

In [4]:
# Synthetic data
torch.manual_seed(42)
batch_size = 3
seq_len = 4
d_model = 8
num_heads = 2

q = torch.rand(batch_size, seq_len, d_model)
k = torch.rand(batch_size, seq_len, d_model)
v = torch.rand(batch_size, seq_len, d_model)
print(q.shape)

device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

torch.Size([3, 4, 8])


In [30]:
class SinusoidalPositionalEmbedding(nn.Module):

    def __init__(self, max_seq_len: int, d_model: int):
        """
        Initializes the sinusoidal positional embedding.

        Args:
            max_seq_len (int): Maximum sequence length.
            d_model (int): Embedding dimension.
        """
        super().__init__()
        # i can go from 0 to d_model / 2
        max_i = d_model // 2  # TODO: test this and increment if needed
        i_vec = torch.arange(max_i)
        wi = torch.pow(10000, 2*i_vec/d_model)

        # pos can range from 0 to max_seq_len
        # We need to compute pe for every i for every pos, so we use broadcasting
        pos_vec = torch.arange(max_seq_len).reshape(-1, 1)

        # pe(pos, 2i)   = sin(pos * wi)
        # pe(pos, 2i+1) = cos(pos * wi)

        pe_sin = torch.sin(pos_vec * wi)  # shape: (max_seq_len, max_i)
        pe_cos = torch.cos(pos_vec * wi)  # shape: (max_seq_len, max_i)

        # Now, we intersprese these
        pe = torch.stack([pe_sin, pe_cos], dim=2)  # shape: (max_seq_len, max_i, 2)
        pe = pe.reshape(max_seq_len, max_i*2)
        pe = pe[:, :d_model]  # chop off last column if not needed
        assert(pe.shape == (max_seq_len, d_model))
        self.register_buffer("pe", pe)


    def forward(self, x):
        """
        Returns the positional embedding for a given input tensor.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len).

        Returns:
            Tensor: Positional embeddings of shape (batch_size, seq_len, d_model).
        """
        batch_size, seq_len = x.shape
        return self.pe[:seq_len, :].unsqueeze(0).tile(batch_size, 1, 1)


In [31]:
from fairseq.modules.sinusoidal_positional_embedding import SinusoidalPositionalEmbedding

max_seq_len = 100
d_model = 64

# Fairseq's implementation requires the number of embeddings (seq length) and embedding dim
pos_emb = SinusoidalPositionalEmbedding(d_model, max_seq_len, padding_idx=None)

# Generate embeddings for a sequence of length 50
seq_len = 50
positions = torch.arange(seq_len).unsqueeze(0)  # Shape: (1, seq_len)
positional_encoding = pos_emb(positions)  # Shape: (1, seq_len, d_model)

custom_pos_emb = SinusoidalPositionalEmbedding(d_model, max_seq_len)
positional_encoding_custom = custom_pos_emb(positions)

# print(positional_encoding_custom.shape)  # (1, 50, 64)

ModuleNotFoundError: No module named 'fairseq'